# Creating a MOCK dataset to make classification and recommendation
Author: Karina Condeixa

CONDITION:
 "available": true

QUESTIONS
- number for views 
- like (with % of views-maximum?)

TODO:
- item_gone_timer = datetime_post - datetime_with_status_available
- item_lat_lng
- check lat_lng_samples.csv


In [315]:
# import packages
import pandas as pd
from faker import Faker, providers
from faker.providers.address.de_DE import Provider as DeDeAddressProvider
from faker.generator import random
from faker.providers import BaseProvider
# import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib notebook
# %matplotlib 

import folium
# from geopy.geocoders import Nominatim

In [316]:
# define global variables

maximun_n_views = 2001
pct_of_likes_over_views = 20
n_records_ranking = 5000
n_records = 5000

limit = '-30d'  # limit of 30 days of item in the app
maximun_n_hours_avalilable = 100

random_seed = 1000


# post_date: something between the last 30 days - limit = '-30d'
# timer = (date.today()) - post_date

In [317]:
category = ['furniture',  'clothes',  'kitchen_utensils', 'kids', 'other'] 
condition = ['poor', 'acceptable', 'good', 'like_new'] 
available = ['1', '0']    

# postcodes = postcodes_berlin_series
    
n_views = []
for v in range(maximun_n_views):  # until 200 views
    n_views.append(v)

pct_likes = []
for l in range(pct_of_likes_over_views):  # until 20% of the n_views
    pct_likes.append(l/100)
 
# datetime post - datetime now, I create a maximum number of hours
n_hours_avalilable = []
for h in range(maximun_n_hours_avalilable):  
    n_hours_avalilable.append(h)
    
distance = []
for d in range(32000):
    distance.append(d)
    


### Postcodes

In [318]:
# immport and clean original dataset removing poostcodes from Germanay, that are not in Berlin
postcodes_de = pd.read_excel(r'data/German-Zip-Codes.xlsx', sheet_name='Berlin')
df = pd.DataFrame(postcodes_de)
df.set_axis(["postcodes_berlin"], axis=1,inplace=True)
df = (df["postcodes_berlin"].str[8:-11])
df.to_csv('data/postcodes_berlin.csv', index=False)

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_2691/2575223029.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(["postcodes_berlin"], axis=1,inplace=True)


### Creating the postcode series

In [319]:
# import postcodes from Berlin and create a dataframe removing indexes and headers
postcodes_berlin = pd.read_csv('data/postcodes_berlin.csv')
print(postcodes_berlin)
postcodes_berlin_series = postcodes_berlin[:][1:].squeeze()

     postcodes_berlin
0               10117
1               10119
2               10178
3               10179
4               10243
..                ...
184             14169
185             14193
186             14195
187             14197
188             14199

[189 rows x 1 columns]


### Creating lists

In [320]:
# Importing Modules
import numpy as np
import random
# Use 'conda install shapely' to import the shapely library.
from shapely.geometry import Polygon, Point
np.random.seed(1000)
random.seed(1000)



### Creating the datasets

In [321]:
# Note: # multi_locale_generator = Faker(['it_IT', 'en_US', 'de-DE', 'pt_BR', 'es-ES', 'fr-FR', 'ru-RU', 'tr-TR'])

# Instantiate Faker with multiple locales
german_locale_generator = Faker(['de_DE'])
fake = Faker()
Faker.seed(0)


### List of latitude and longitude co-ordinates 
Based on a sample of co-ordinates from [free data Berlin](https://daten.berlin.de/datensaetze/stra%C3%9Fenverkehrsunf%C3%A4lle-nach-unfallort-berlin-2021) for Road traffic accidents by accident location in Berlin 2021.


In [322]:
lat_lng_samples = pd.read_csv("data/lat_lng_samples.csv")
lat_lng_samples.head(2)

,lat,lng
0,"5,253,393,955","1,342,689,483"
1,"5,243,995,086","1,339,209,027"


In [323]:
# replace comma to dot
lat_lng_samples = lat_lng_samples.apply(lambda x: x.str.replace(',','.'))

In [324]:
list_lat = lat_lng_samples.lat.values.tolist()
list_lng = lat_lng_samples.lng.values.tolist()
lat_lng_samples.head(2)


,lat,lng
0,5.253.393.955,1.342.689.483
1,5.243.995.086,1.339.209.027


In [325]:
lat_lng_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     11192 non-null  object
 1   lng     11192 non-null  object
dtypes: object(2)
memory usage: 175.1+ KB


### Item dataset

In [326]:
# An initial classification that is not being used to normalize with UXs categories. But it is saved for future updates.
# category_item = ['furniture-sofa',
#                  'furniture-armchair',
#                  'furniture-chair',
#                  'furniture-table',
#                  'furniture-desk',
#                  'furniture-bed',
#                  'furniture-bookcase',
#                  'furniture-bedside_table',
#                  'furniture-cabinet',
#                  'furniture-wardrobe',
#                  'furniture-shelf',
#                  'furniture-cupboard',
#                  'furniture-rollcontainers',
#                  'furniture-shoe_rack',
#                  'furniture-mirror',
#                  'furniture-cot',
#                  'furniture-trolley',
#                  'appliance-washing_machine',
#                  'appliance-dish_washer',
#                  'appliance-drying_rack',
#                  'appliance-refrigerator',
#                  'appliance-blender',
#                  'appliance-extractor_hood',
#                  'appliance-clothes_iron',
#                  'appliance-vacuum_cleaner',
#                  'appliance-sandwich_maker',
#                  'appliance-kettle',
#                  'appliance-air_conditioner',
#                  'appliance-heater',
#                  'appliance-pan',
#                  'appliance-popcorn_maker',
#                  'appliance-coffee_machine',
#                  'appliance-stove',
#                  'lighting-lighting',
#                  'lighting-chandelier',
#                  'lighting-lightbulb',
#                  'musical_equipment-guitar',
#                  'musical_equipment-sound_amplifier',
#                  'musical_equipment-contrabass',
#                  'musical_equipment-battery',
#                  'musical_equipment-piano',
#                  'tech-desktop',
#                  'tech-laptop',
#                  'tech-phone',
#                  'tech-keyboard',
#                  'clothes-woman_jacket',
#                  'clothes-man_jacket',
#                  'clothes-child_jacket',
#                  'clothes-woman_clothes',
#                  'clothes-man_clothes',
#                  'clothes-child_clothes',
#                  'shoes-woman_shoes',
#                  'shoes-man_shoes',
#                  'shoes-child_shoes',
#                  'miscelaneaous-ironing_board',
#                  'miscelaneaous-picture_frame',
#                  'miscelaneaous-bicycle',
#                  'miscelaneaous-plant',
#                  'miscelaneaous-carpet',
#                  'miscelaneaous-roller_skates',
#                  'miscelaneaous-ski_skates',
#                  'miscelaneaous-books',
#                  'miscelaneaous-purse',
#                  'miscelaneaous-suitcase',
#                  'miscelaneaous-shopping_venture',
#                  'miscelaneaous-board',
#                  'miscelaneaous-frame',
#                  'home-mattress', 
#                  'home-carpet',
#                  'kids-stroller',
#                  'kids-baby_carriage']

In [327]:
# addresses = []
# for _ in range(50000000):
#     address = german_locale_generator.address()
#     if 'Berlin' in address:
#         addresses.append(address)
#     else:
#         if len(addresses) < 1500: 
#             continue
#         else: 
#             break
            
# df['is_berlin'] = df['address'].apply(lambda x: 1 if 'Berlin' in x else 0)

In [328]:

#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

#         item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')

In [329]:
# # Define the desired polygon : points choosen in Google maps

# poly = Polygon([(52.645883, 13.395869), 
#                 (52.526568, 13.645808),
#                 (52.381789, 13.405482),
#                 (52.484773, 13.136317)])


# min_x = 52.381789
# max_x = 52.645883
# min_y = 13.136317
# max_y = 13.645808

# # Defining the randomization generator
# def polygon_random_points (poly, num_records):
#     min_x, min_y, max_x, max_y = poly.bounds
#     points = []
#     while len(points) < num_records:
#         random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
#         if (random_point.within(poly)):
#             points.append(random_point)
#     return points
    # Choose the number of points desired. T\ 
# points = polygon_random_points(poly,n_records)
# # Testing the results.
# for p in points:
#     print(p.x,",",p.y)


In [330]:
# from math import sin, cos, sqrt, atan2, radians

# # Approximate radius of earth in km
# R = 6373.0

# lat1 = radians(52.2296756)
# lon1 = radians(21.0122287)
# lat2 = radians(52.406374)
# lon2 = radians(16.9251681)

# dlon = lon2 - lon1
# dlat = lat2 - lat1

# a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
# c = 2 * atan2(sqrt(a), sqrt(1 - a))

# distance = R * c

# print("Result: ", distance)
# print("Should be: ", 278.546, "km")

In [358]:

def data_classification_1(num_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)  
        item[i]['item_condition'] = np.random.choice(condition)

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

        item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        item[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
        
        item[i]['item_available'] = np.random.choice(available)
             
    return item

In [359]:
ml_class_df = pd.DataFrame(data_classification_1(n_records)).transpose()
ml_class_df.head()

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available
0,furniture,like_new,13583,88,29731,981,186,0
1,furniture,good,14057,46,5382,1107,210,0
2,kitchen_utensils,good,10829,60,2865,1310,91,0
3,furniture,acceptable,10435,67,431,1897,56,1
4,kids,acceptable,14109,90,15303,1280,89,1


In [360]:
# ml_class_df['item_id'] = ml_class_df.index +1  # add item_id
# item_id_series = ml_class_df['item_id']  # storage in a variable to use later

# ml_class_df.head(10)

In [361]:
# def search(num_records): 
  
#     # dictionary 
#     search ={} 
#     for i in range(0, num_records): 
#         search[i] = {}
# #         item[i]['item_status'] = 1
#         search[i]['item_category'] = np.random.choice(category)
#         search[i]['item_id'] = np.random.choice(range(1, n_records))  
# #         search[i]['local'] = np.random.choice(XXXXX)
# #         search[i]['time'] = np.random.choice(xxx)

#     return search

In [362]:
### User

In [363]:
# user_df = item_df.copy()

In [364]:
# user_df = user_df.rename(columns={"item_id": "user_id"})

In [365]:
# user_df.head(10)

In [366]:
# # item_df.get('item_timer')

# item_df['item_timer'] = item_df['item_timer'].astype(str).str[:2].str.strip()
# item_df['item_timer'] = item_df['item_timer'].astype(int)

# item_df = item_df.rename(columns={'item_timer': 'item_timer_days'})

# # item_df.drop('item_timer', axis=1)
# item_df.info()

# item_df.head()

In [367]:
# # saves csv files
# ml_class_df.to_csv('data/ml_class_df.csv', index=False)
# # user_df.to_csv('data/user_df.csv', index=False)

In [368]:
def create_data_rankings(num_records_ranking): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records_ranking): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)
        item[i]['item_condition'] = np.random.choice(condition)

        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now') # in days, can be improved to 
        
        item[i]['distance'] = np.random.choice(distance)
        
        
 
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
              
    return item

In [369]:
ranking_df = pd.DataFrame(create_data_rankings(n_records_ranking)).transpose()
ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes
0,other,like_new,59,5988,1776,230
1,kitchen_utensils,acceptable,51,25010,15,0
2,clothes,good,6,10324,1639,32
3,furniture,acceptable,93,17227,1763,282
4,other,poor,4,30836,642,0


In [370]:
ranking_df['item_id'] = ranking_df.index +1  # add item_id
item_id_series = ranking_df['item_id']  # storage in a variable to use later

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,other,like_new,59,5988,1776,230,1
1,kitchen_utensils,acceptable,51,25010,15,0,2
2,clothes,good,6,10324,1639,32,3
3,furniture,acceptable,93,17227,1763,282,4
4,other,poor,4,30836,642,0,5


In [371]:
# ranking_df['item_condition']
# ['poor', 'acceptable', 'Good', 'like_new'] 

ranking_df['item_condition'] = ranking_df.item_condition.replace(['like_new', 'good', 'acceptable', 'poor'], [1, 2, 3, 4])

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,other,1,59,5988,1776,230,1
1,kitchen_utensils,3,51,25010,15,0,2
2,clothes,2,6,10324,1639,32,3
3,furniture,3,93,17227,1763,282,4
4,other,4,4,30836,642,0,5


In [372]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_category         5000 non-null   object
 1   item_condition        5000 non-null   int64 
 2   item_available_timer  5000 non-null   object
 3   distance              5000 non-null   object
 4   n_views               5000 non-null   object
 5   n_likes               5000 non-null   object
 6   item_id               5000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 312.5+ KB


In [373]:
# saves csv files
ranking_df.to_csv('../recommendation/data/ranking_df.csv', index=False)

In [374]:
ml_class_df.to_csv('data/ml_class_df.csv', index=False)

In [375]:
ml_class_df.head()

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available
0,furniture,like_new,13583,88,29731,981,186,0
1,furniture,good,14057,46,5382,1107,210,0
2,kitchen_utensils,good,10829,60,2865,1310,91,0
3,furniture,acceptable,10435,67,431,1897,56,1
4,kids,acceptable,14109,90,15303,1280,89,1


In [376]:
encoded = pd.read_csv('data/item_data_encoded.csv',index_col=None)
encoded.head(14)

,item_category,item_condition,item_available_timer,n_views,n_likes,item_available
0,4,1,64,1275,12,0
1,1,3,40,1897,75,1
2,3,2,20,498,44,1
3,3,2,30,244,31,1
4,5,3,30,1550,77,1
5,4,4,32,1972,354,0
6,3,4,16,1268,139,0
7,4,1,31,158,20,1
8,2,1,27,47,7,0
9,4,4,55,600,48,1


In [377]:
# review later, I used a file made in excel

n_records_new = 15

def new_data(n_records_new): 
  
    # dictionary 
    new_data ={} 
    for i in range(0, n_records_new): 
        new_data[i] = {}
#         item[i]['item_status'] = 1
        new_data[i]['item_category'] = np.random.choice(category)
#         new_data[i]['item_id'] = np.random.choice(range(1, n_records_new))  
        new_data[i]['item_condition'] = np.random.choice(condition)

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

#         new_data[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        new_data[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        new_data[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        new_data[i]['n_views'] = np.random.choice(n_views)
        new_data[i]['n_likes'] = int(new_data[i]['n_views'] * (np.random.choice(pct_likes)))
        new_data[i]['item_available'] = np.random.choice(available)
        
             
    return new_data

new_data(n_records_new)

{0: {'item_category': 'furniture',
  'item_condition': 'good',
  'item_available_timer': 25,
  'distance': 8636,
  'n_views': 362,
  'n_likes': 18,
  'item_available': '0'},
 1: {'item_category': 'furniture',
  'item_condition': 'poor',
  'item_available_timer': 86,
  'distance': 21381,
  'n_views': 453,
  'n_likes': 72,
  'item_available': '1'},
 2: {'item_category': 'other',
  'item_condition': 'good',
  'item_available_timer': 71,
  'distance': 5967,
  'n_views': 1329,
  'n_likes': 39,
  'item_available': '1'},
 3: {'item_category': 'clothes',
  'item_condition': 'like_new',
  'item_available_timer': 84,
  'distance': 11580,
  'n_views': 742,
  'n_likes': 81,
  'item_available': '1'},
 4: {'item_category': 'clothes',
  'item_condition': 'poor',
  'item_available_timer': 32,
  'distance': 10362,
  'n_views': 1603,
  'n_likes': 32,
  'item_available': '0'},
 5: {'item_category': 'clothes',
  'item_condition': 'good',
  'item_available_timer': 98,
  'distance': 27597,
  'n_views': 1906

In [378]:
new_data= pd.DataFrame(new_data(n_records_new)).transpose()
new_data.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,kids,good,26,25120,853,102,1
1,other,like_new,7,23070,971,29,1
2,other,poor,19,7048,167,5,1
3,other,acceptable,93,6666,1834,146,0
4,kids,poor,76,16248,981,0,0


In [382]:
new_data.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,kids,good,26,25120,853,102,1
1,other,like_new,7,23070,971,29,1
2,other,poor,19,7048,167,5,1
3,other,acceptable,93,6666,1834,146,0
4,kids,poor,76,16248,981,0,0


In [383]:
# new_data.to_csv('data/new_data.csv', index=False) # I had an error and made in excel